# Reduceret echelonform

In [10]:
import numpy as np
from sympy import Matrix, latex

def row_echelon_form_operations(matrix):
    # Konverter matricen til et Sympy matrix objekt
    sympy_matrix = Matrix(matrix)

    # Konverter Sympy matrix til et NumPy array for numeriske operationer
    numpy_array = np.array(sympy_matrix.tolist(), dtype=float)

    # Følg row operationer til senere at få dem
    operations = []

    # Anvend Gaussian elimination til at få reduceret row echelon form
    num_rows, num_cols = numpy_array.shape
    lead = 0
    for r in range(num_rows):
        if lead >= num_cols:
            break

        i = r
        while numpy_array[i, lead] == 0:
            i += 1
            if i == num_rows:
                i = r
                lead += 1
                if num_cols == lead:
                    break

        if i != r:
            numpy_array[[i, r], :] = numpy_array[[r, i], :]
            operations.append(f"Swap R{i+1} and R{r+1}")

        if numpy_array[r, lead] != 0:
            pivot = numpy_array[r, lead]
            numpy_array[r, :] /= pivot
            operations.append(f"Scale R{r+1} by 1/{pivot}")

        for i in range(num_rows):
            if i != r:
                factor = numpy_array[i, lead]
                numpy_array[i, :] -= factor * numpy_array[r, :]
                operations.append(f"R{i+1} -> R{i+1} - {factor} * R{r+1}")

        lead += 1

    # Konverter NumPy array til Sympy matrix
    echelon_matrix = Matrix(numpy_array)

    # Konverter matricen til latex format
    echelon_latex = latex(echelon_matrix)

    return echelon_latex, operations

# Matric som skal reduceres (hint: hvis du skal finde echelonform til et ligningssystem, så indsæt matricen du får EFTER at bruge np.stack)
matrix = [[1, 1, 2, 1, 0],
          [1, 1, 3, 2, 1],
          [2, 2, 5, 4, 1]]

echelon_form, operations = row_echelon_form_operations(matrix)
print("Row Echelon Form:")
print(echelon_form)
print("Operations:")
for operation in operations:
    print(operation)

Row Echelon Form:
\left[\begin{matrix}1.0 & 1.0 & 0 & 0 & -2.0\\0 & 0 & 1.0 & 0 & 1.0\\0 & 0 & 0 & 1.0 & 0\end{matrix}\right]
Operations:
Scale R1 by 1/1.0
R2 -> R2 - 1.0 * R1
R3 -> R3 - 2.0 * R1
Scale R2 by 1/1.0
R1 -> R1 - 2.0 * R2
R3 -> R3 - 1.0 * R2
Scale R3 by 1/1.0
R1 -> R1 - -1.0 * R3
R2 -> R2 - 1.0 * R3


I forhold til resultaterne ovenfor, så er følgende betydningen: \\
Scale betyder skalar multiplikation (matrix[0, :] *= 1/1.0) \\
R3 -> R3 - 2.0 * R1 betyder R3 skal laves om på med hensyn det til på højre side
Swap betyder der skal byttes om på de to rækker (matrix[ [1, 2], :] = matrix[ [2, 1], :])

In [15]:
# Det er vigtigt at tilføje dtype=float, ellers fungerer 1/1.0 ikke
matrix = np.array([[1, 1, 2, 1, 0], [1, 1, 3, 2, 1], [2, 2, 5, 4, 1]], dtype=float)

matrix[0, :] *= 1/1.0
matrix[1, :] = matrix[1, :] - 1*matrix[0, :]
matrix[2, :] = matrix[2, :] - 2*matrix[0, :]
matrix[1, :] *= 1/1.0
matrix[0, :] = matrix[0, :] - 2*matrix[1, :]
matrix[2, :] = matrix[2, :] - 1*matrix[1, :]
matrix[2, :] *= 1/1.0
matrix[0, :] = matrix[0, :] - (-2)*matrix[2, :]
matrix[1, :] = matrix[1, :] - 1*matrix[2, :]
print(matrix)

[[ 1.  1.  0.  1. -2.]
 [ 0.  0.  1.  0.  1.]
 [ 0.  0.  0.  1.  0.]]
